# Convert to a standardized version of dataframe

A converter to convert data frame to a standard format for carpoolsim:
1. Traffic network links
2. Traffic network nodes
3. Traffic TAZs (polygons)
4. Traffic demands 

In [2]:
import time
import copy
import os
import sys

import numpy as np
import pandas as pd
import geopandas as gpd

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/%autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
pd.options.display.max_columns = None  # display all columns

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# need to set this environmental path everytime you moves the project root folder
os.environ['project_root'] = '/Users/geekduck/Dropbox/gt_survey'

sys.path.append(
    os.environ['project_root']
)

In [5]:
from carpoolsim.basic_settings import *

In [6]:
# load traffic network data
taz_centriod_nodes = gpd.read_file(os.environ["taz"])
df_nodes_raw = gpd.read_file(os.environ['network_nodes'])
df_links_raw = gpd.read_file(os.environ['network_links'])

In [8]:
from carpoolsim.dataclass.traffic_network import (
    TrafficNetworkLink,
    TrafficNetworkNode,
    TrafficAnalysisZone,
)

In [10]:
df_nodes_raw.sample(2)

,N,X,Y,STATION,STAFLAG,PNR,PNR_MAJOR,GEOMETRYSO,lat,lon,geometry
7176,9166,2.280151e+06,1.381880e+06,None,0,0,0,1,33.79887,-84.22076,POINT (-84.22076 33.79887)
15714,32700,2.124737e+06,1.313745e+06,None,0,0,0,1,33.61034,-84.73112,POINT (-84.73112 33.61034)


In [15]:
# clean node object
node_name_map = {
    "N": "nid",
    "lat": "lat",
    "lon": "lon",
    "X": "x",
    "Y": "y"
}
df_nodes_raw = df_nodes_raw.rename(columns=node_name_map)

df_nodes_lst = []
for index, row in df_nodes_raw.iterrows():
    df_nodes_lst.append(
        TrafficNetworkNode(row["nid"], row["lon"], row["lat"], row["x"], row["y"])
    )

In [9]:
df_links_raw.sample(2)

,A,B,SHAPE_LENGT,DISTANCE,NAME,COUNTY,FACTYPE,TOLLID,LANES,LANESEA,LANESAM,LANESMD,LANESPM,LANESEV,AUXLANE,PROHIBITION,TRNDIST,TRNTIME,FCLASS,MINBUSSPD,MAXBUSSPD,TWOWAY,CNTSTATION,DIRAADT00,DIRAADT05,DIRAADT08,DIRAADT10,DIRAWDT00,DIRAWDT05,DIRAWDT08,DIRAWDT10,ATR,ATR_DIR,HR1,HR2,HR3,HR4,HR5,HR6,HR7,HR8,HR9,HR10,HR11,HR12,HR13,HR14,HR15,HR16,HR17,HR18,HR19,HR20,HR21,HR22,HR23,HR24,CNTFAC,FACTOR,PSPEED04,MEDIAN,SHOULDER,CONTROL,STRATEGIC,ID,CMSID,SIGNALIMPR,SPEED_LIMI,FWYSEG,ARTSEG,SCREENLINE,NOTES,CNTFIX,HOVMERGE,GPID,WEAVEFLAG,TMC,TMCFLAG,TMCDIST,EAOBS,AMOBS,MDOBS,PMOBS,EVOBS,EATIME,AMTIME,MDTIME,PMTIME,EVTIME,EASPD,AMSPD,MDSPD,PMSPD,EVSPD,RAMPFLAG,GEOMETRYSOU,geometry
20174,10811,10812,1816.37683,0.34401,SR 138,247,10,0,2,0,0,0,0,0,0,0,0.0,0.0,14,0,0,0,0,0,0,0,0,0,0,0,0,0,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,55,t,n,n,37,TMP-RO-04,247-08-013,0,0,0,0,0,None,0,0,0,0,None,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,"LINESTRING (2351287.240 1334878.790, 2352788.3..."
29197,14979,10580,3302.03809,0.62539,Johnson Ferry Rd,67,10,0,2,0,0,0,0,0,0,0,0.0,0.0,16,0,0,0,0,0,0,0,0,0,0,0,0,0,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,30,t,n,n,16,None,None,0,0,0,0,0,None,0,0,0,0,None,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,"LINESTRING (2217667.994 1451119.201, 2215594.6..."


In [19]:
df_links_raw["a"].dtype

dtype('int64')

In [20]:
# clean link object
link_name_map = {
    "A": "a",
    "B": "b",
    "DISTANCE": "distance",
    "geometry": "geometry",
}
df_links_raw = df_links_raw.rename(columns=link_name_map)
df_links_raw["a_b"] = f'{df_links_raw["a"]}_{df_links_raw["b"]}'


df_links_lst = []
for index, row in df_links_raw.iterrows():
    df_nodes_lst.append(
        TrafficNetworkLink(row["a"], row["b"], row["a_b"], row["distance"], row["geometry"])
    )

In [21]:
taz_centriod_nodes.sample(2)

,OBJECTID_1,OBJECTID,MTAZ10,CTAZ10,TRACT10,TAD10,SD10,FIPS,COUNTY,ACRES,GlobalID,last_edite,Shape_Leng,Shape_Area,TAZS,TAZS_int,layer,path,geometry
5570,5053,5053,5053,13045002,1.304591e+10,13045045,124,13045,Carroll,5656.712038,{00B5DA3F-0EA1-4553-986E-0B11FC500BD1},None,0.207761,0.002223,5053,5053,None,None,"POLYGON ((-84.90683 33.56988, -84.90646 33.569..."
2151,453,453,453,13121279,1.312101e+10,13121071,5,13121,Fulton,47.746530,{47C142D8-00C2-4C84-9282-1B29FCBAE01F},None,0.019094,0.000019,453,453,TAZ_zone3,/Users/diyi93/Dropbox/My Mac (Diyiâs MacBook...,"POLYGON ((-84.43295 33.75599, -84.43355 33.755..."


In [ ]:
# clean taz object
taz_name_map = {
    "OBJECTID": "taz_id",
    "geometry": "geometry",
}
